### Experiment
This notebook is used for tuning the hyper-parameters of pretrained models.


In [1]:
# import method that can run model with option

import logging
import os
import time

import pandas as pd
import torch

# 增加可读性
from thop import profile, clever_format
import models as module_arch
import utils.data_loader as module_data
import utils.loss_helper as module_loss
import utils.metric as module_metric
from run_model import run
from utils.parse_config import ConfigParser
from utils.tools import prepare_device
# fix random seeds for reproducibility
from utils.trainer import Trainer

In [2]:
# define the hyper-parameters here
image_sizes = [128, 256, 512]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64]

In [3]:
# define default training option here
option = {
    "name": "googlenet",
    "arch;args;pretrained_model": "googlenet", # pretrained model name
    "arch;args;freeze_param": 1, # freeze the parameters of pretrained model
    "data_loader;args;img_size": 512,
    "data_loader;args;batch_size": 32,
    "optimizer;args;lr": 0.001
}

In [3]:
# define evaluation result path
result_path = "saved/result"
os.makedirs(result_path, exist_ok=True)


In [7]:
trainers = []
for image_size in image_sizes:
    option.update({"data_loader;args;img_size": image_size})
    run_id = f"{image_size}_{option['data_loader;args;batch_size']}_{option['optimizer;args;lr']}"
    # close old logging process
    logging.shutdown()
    trainers.append(run("pretrained_model.json", option, run_id, False))

100%|██████████| 20/20 [00:11<00:00,  1.76it/s]


In [20]:
def evaluate_result(train_results):
    # define the dataframe of result
    result_df = pd.DataFrame(columns=["model", "val_auc", "val_acc", "val_loss", "hyper-parameters"])

    for trainer in train_results:
        config = trainer.config
        log = trainer.log_best
        param = f"{config['data_loader']['args']['img_size']}_{config['data_loader']['args']['batch_size']}_{config['optimizer']['args']['lr']}"
        series = pd.Series({
            "model": config["name"], "val_auc": log["val_auc"], "val_acc": log["val_accuracy"],
            "hyper-parameters": param, "val_loss": log["val_loss"]
        })
        result_df = result_df.append(series, ignore_index=True)
    return result_df

In [22]:
image_size_result = evaluate_result(trainers)
image_size_result

,model,val_auc,val_acc,val_loss,hyper-parameters
0,googlenet,0.6749,0.6322,0.6478,128_32_0.001
1,googlenet,0.7291,0.6651,0.6180,256_32_0.001
2,googlenet,0.7373,0.6541,0.6152,512_32_0.001


In [27]:
# I choose image_size=256, because the time consuming is much lower then image_size=512, while the performance is still good enough.
option.update({"data_loader;args;img_size": 256})
trainers = []
for bs in batch_sizes:
    option.update({"data_loader;args;batch_size": bs})
    run_id = f"{option['data_loader;args;img_size']}_{bs}_{option['optimizer;args;lr']}"
    logging.shutdown()
    trainers.append(run("pretrained_model.json", option, run_id))

100%|██████████| 10/10 [00:04<00:00,  2.11it/s]


In [28]:
bs_result = evaluate_result(trainers)
bs_result

,model,val_auc,val_acc,val_loss,hyper-parameters
0,googlenet,0.7315,0.6573,0.6178,256_16_0.001
1,googlenet,0.7291,0.6651,0.6180,256_32_0.001
2,googlenet,0.7363,0.6761,0.6125,256_64_0.001


In [29]:
# update with best batch_size: a larger batch size performs better
option.update({"data_loader;args;batch_size": 64})
trainers = []
for lr in learning_rates:
    option.update({"optimizer;args;lr": lr})
    run_id = f"{option['data_loader;args;img_size']}_{option['data_loader;args;batch_size']}_{lr}"
    logging.shutdown()
    trainers.append(run("pretrained_model.json", option, run_id))

100%|██████████| 10/10 [00:05<00:00,  1.78it/s]


In [30]:
lr_result = evaluate_result(trainers)
lr_result

,model,val_auc,val_acc,val_loss,hyper-parameters
0,googlenet,0.6447,0.5947,0.7154,256_64_0.01
1,googlenet,0.7363,0.6761,0.6125,256_64_0.001
2,googlenet,0.7500,0.7027,0.6025,256_64_0.0001


In [31]:
# update with best learning rate
option.update({"data_loader;args;lr": 0.0001})

[]


Train ResNet, VGG, and DenseNet, and evaluate the results.


In [4]:
# define best suitable training option here
option = {
    "arch;args;pretrained_model": "googlenet", # pretrained model name
    "arch;args;freeze_param": 1, # freeze the parameters of pretrained model
    "data_loader;args;img_size": 256,
    "data_loader;args;batch_size": 64,
    "optimizer;args;lr": 0.0001
}
resnet_models = ["resnet34", "resnet50", "resnet101", "resnet152"]
vgg_models = ["vgg11_bn", "vgg13_bn", "vgg16_bn", "vgg19_bn"]
densnet_models = ["densenet121", "densenet169", "densenet201", "densenet161"]
all_models = resnet_models+  vgg_models + densnet_models

In [33]:
trainers = []
for model in all_models[:5]:
    print(f"Run model: {model}")
    option.update({"name": model, "arch;args;pretrained_model": model})
    logging.shutdown()
    trainers.append(run("pretrained_model.json", option, model))

Run model: resnet34

Run model: resnet50
Run model: resnet101

Run model: resnet152
Run model: vgg11_bn

Run model: vgg13_bn



Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\resnet34-333f7ec4.pth
100%|██████████| 10/10 [00:06<00:00,  1.52it/s]
Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\resnet101-5d3b4d8f.pth
100%|██████████| 10/10 [00:10<00:00,  1.10s/it]
Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\vgg11_bn-6002323d.pth
100%|██████████| 10/10 [00:07<00:00,  1.30it/s]
Downloading: "https://download.pytorch.org/models/vgg13_bn-abd245e5.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\vgg13_bn-abd245e5.pth
  0%|          | 0/88 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 6.00 GiB total capacity; 2.76 GiB already allocated; 458.44 MiB free; 3.60 GiB reserved in total by PyTorch)

In [ ]:
for trainer in trainers:
    trainer.model.cpu()
torch.cuda.empty_cache()

In [8]:
option = {
    "arch;args;pretrained_model": "googlenet", # pretrained model name
    "arch;args;freeze_param": 1, # freeze the parameters of pretrained model
    "data_loader;args;img_size": 256,
    "data_loader;args;batch_size": 32,
    "optimizer;args;lr": 0.0001
}
trainers = []
for model in all_models[5:]:
    print(f"Run model: {model}")
    option.update({"name": model, "arch;args;pretrained_model": model})
    logging.shutdown()
    trainers.append(run("pretrained_model.json", option, model))



Run model: vgg13_bn
Run model: vgg16_bn

Run model: vgg19_bn
Run model: densenet121

Run model: densenet169

Run model: densenet201

Run model: densenet161



100%|██████████| 20/20 [00:10<00:00,  1.87it/s]
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth
100%|██████████| 20/20 [00:14<00:00,  1.33it/s]
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 20/20 [00:08<00:00,  2.48it/s]
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\densenet169-b2777c0a.pth
100%|██████████| 20/20 [00:08<00:00,  2.31it/s]
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\densenet201-c1103571.pth
100%|██████████| 20/20 [00:09<00:00,  2.05it/s]
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to C:\Users\Rui/.cache\torch\hub\checkpoints\densenet161-8d451a50.pth
100%|██████████| 20/20 [00:11<00:00,

In [2]:
# run with larger image size, the batch size can be too large
option = {
    "arch;args;pretrained_model": "googlenet", # pretrained model name
    "arch;args;freeze_param": 1, # freeze the parameters of pretrained model
    "data_loader;args;img_size": 512,
    "data_loader;args;batch_size": 32,
    "optimizer;args;lr": 0.0001
}
# run with dropout=0.2
trainers_512 = []
for model in ["resnet101", "vgg11_bn", "densenet169"]:
    print(f"Run model: {model}")
    option.update({"name": "best_model", "arch;args;pretrained_model": model})
    logging.shutdown()
    trainers_512.append(run("pretrained_model.json", option, f"{model}"))

Run model: resnet101
Run model: vgg11_bn


  0%|          | 0/176 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 2.00 GiB (GPU 0; 6.00 GiB total capacity; 2.76 GiB already allocated; 1.27 GiB free; 2.79 GiB reserved in total by PyTorch)

In [5]:

def run_test(conf, test_model):
    # setup data_loader instances
    data_loader = conf.init_obj("data_loader", module_data)

    # prepare for (multi-device) GPU training
    device, device_ids = prepare_device(conf["n_gpu"])
    test_model = test_model.to(device)
    if len(device_ids) > 1:
        test_model = torch.nn.DataParallel(test_model, device_ids=device_ids)

    # get function handles of loss and metrics
    criterion = getattr(module_loss, conf["loss"])
    metrics = [getattr(module_metric, met) for met in conf["metrics"]]

    # build optimizer, learning rate scheduler. delete every lines containing lr_scheduler for disabling scheduler
    trainable_params = filter(lambda p: p.requires_grad, test_model.parameters())
    optimizer = conf.init_obj("optimizer", torch.optim, trainable_params)
    lr_scheduler = conf.init_obj("lr_scheduler", torch.optim.lr_scheduler, optimizer)

    # define trainer of model
    test_trainer = Trainer(test_model, criterion, metrics, optimizer,
                      config=conf,
                      device=device,
                      data_loader=data_loader.train_loader,
                      valid_data_loader=data_loader.valid_loader,
                      lr_scheduler=lr_scheduler)
    return test_trainer._valid_epoch(0)


In [19]:
evaluation_df = pd.DataFrame(columns=["model", "val_auc", "val_acc", "test_auc", "test_acc", "hyper_parameters", "parameters", "FLOPS", "Time Consuming"])
# load the statistic information
for path in os.scandir("saved/models/googlenet"):
    hyper_parameters = path.name
    ckpt_path = os.path.join(path, "model_best.pth")
    checkpoint = torch.load(ckpt_path, map_location=torch.device('cuda'))
    log_dic = checkpoint["log"]
    checkpoint["config"]["name"] = "test"
    config = ConfigParser(checkpoint["config"], modification={"data_loader;args;valid_df_path": "dataset/test_df.csv"})
    model = config.init_obj("arch", module_arch)
    model.load_state_dict(checkpoint["state_dict"])
    start_time = time.time()
    test_log = run_test(config, model)
    time_consume = round(time.time() - start_time, 2)

    img_size, bs, lr = path.name.split("_")
    flops, params = profile(model, inputs=(torch.randn(int(bs), 3, int(img_size), int(img_size)).to(torch.device("cuda")), ))
    flops, params = clever_format([flops, params], "%.3f")
    line = pd.Series({
    "model": "GoogLeNet", "val_auc": log_dic["val_auc"], "val_acc": log_dic["val_accuracy"],
        "hyper_parameters": path.name, "parameters": params, "FLOPS": flops,
        "test_auc": test_log["auc"], "test_acc": test_log["accuracy"], "Time Consuming": f"{time_consume} s"
    })
    evaluation_df = evaluation_df.append(line, ignore_index=True)

100%|██████████| 20/20 [00:09<00:00,  2.04it/s]


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torchvision.models.googlenet.BasicConv2d'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchvision.models.googlenet.Inception'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.googlenet.GoogLeNet'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.mod

In [20]:
evaluation_df.to_csv(os.path.join(result_path, "hyper_tuning.csv"), index=False)
evaluation_df

,model,val_auc,val_acc,test_auc,test_acc,hyper_parameters,parameters,FLOPS,Time Consuming
0,GoogLeNet,0.6749,0.6322,0.6714,0.6338,128_32_0.001,6.753M,15.751G,3.4 s
1,GoogLeNet,0.7315,0.6573,0.7588,0.7011,256_16_0.001,6.753M,31.446G,4.58 s
2,GoogLeNet,0.7291,0.6651,0.7314,0.6729,256_32_0.001,6.753M,62.892G,4.5 s
3,GoogLeNet,0.7500,0.7027,0.7499,0.6823,256_64_0.0001,6.753M,125.784G,4.45 s
4,GoogLeNet,0.7363,0.6761,0.7356,0.6729,256_64_0.001,6.753M,125.784G,4.51 s
5,GoogLeNet,0.6447,0.5947,0.6294,0.5900,256_64_0.01,6.753M,125.784G,4.43 s
6,GoogLeNet,0.7373,0.6541,0.7417,0.6714,512_32_0.001,6.753M,251.457G,9.91 s


In [17]:
pretrained_compare_df = pd.DataFrame(columns=["model", "val_auc", "val_acc", "test_auc", "test_acc", "parameters", "FLOPS", "Time Consuming"])
for model_name in all_models:
    log_dir = os.path.join("saved/models", model_name)
    ckpt_path = os.path.join(log_dir, os.listdir(log_dir)[-1], "model_best.pth")
    checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))
    log_dic = checkpoint["log"]

    checkpoint["config"]["name"] = "test"
    config = ConfigParser(checkpoint["config"], modification={"data_loader;args;valid_df_path": "dataset/test_df.csv"})
    model = config.init_obj("arch", module_arch)
    model.load_state_dict(checkpoint["state_dict"])
    start_time = time.time()
    test_log = run_test(config, model)
    time_consume = round(time.time() - start_time, 2)

    img_size, bs = 256, 32
    model = model.cpu()
    flops, params = profile(model, inputs=(torch.randn(int(bs), 3, int(img_size), int(img_size)), ))
    flops, params = clever_format([flops, params], "%.3f")
    line = pd.Series({
    "model": model_name, "val_auc": log_dic["val_auc"], "val_acc": log_dic["val_accuracy"],
        "parameters": params, "FLOPS": flops, "test_auc": test_log["auc"], "test_acc": test_log["accuracy"],
        "Time Consuming": f"{time_consume}s"
    })
    torch.cuda.empty_cache()
    pretrained_compare_df = pretrained_compare_df.append(line, ignore_index=True)

100%|██████████| 20/20 [00:08<00:00,  2.26it/s]


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.ResNet'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.Softmax'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.pretrained_model.Pret

In [18]:
pretrained_compare_df.to_csv(os.path.join(result_path, "pretrained_model_compare.csv"), index=False)
pretrained_compare_df

,model,val_auc,val_acc,test_auc,test_acc,parameters,FLOPS,Time Consuming
0,resnet34,0.7330,0.6635,0.7795,0.7089,21.926M,153.443G,5.42s
1,resnet50,0.7165,0.6541,0.7639,0.6854,25.685M,171.829G,5.59s
2,resnet101,0.7441,0.6557,0.7645,0.6948,44.678M,327.412G,7.02s
3,resnet152,0.7357,0.6573,0.7619,0.7027,60.321M,483.096G,8.71s
4,vgg11_bn,0.7389,0.6635,0.7554,0.6839,132.997M,317.755G,5.85s
5,vgg13_bn,0.7280,0.6510,0.7524,0.6776,133.182M,472.977G,7.4s
6,vgg16_bn,0.7265,0.6588,0.7732,0.7089,138.494M,647.087G,8.0s
7,vgg19_bn,0.7179,0.6322,0.7520,0.6792,143.807M,821.197G,8.49s
8,densenet121,0.7419,0.6604,0.7647,0.6870,8.107M,119.767G,6.26s
9,densenet169,0.7683,0.6980,0.7760,0.7074,14.278M,142.013G,6.8s
